In [1]:
from docx import Document
from docx.shared import Inches
from datetime import datetime
from datetime import timedelta
from google.oauth2 import service_account
import os
from google.cloud import bigquery

In [2]:
os.path.abspath(os.getcwd())

'C:\\Users\\Francesco\\Desktop\\Capstone\\codes'

In [21]:
print('Credendtials from environ: {}'.format(os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')))

Credendtials from environ: C:\Users\Francesco\Desktop\Capstone\codes\credentials.json


In [20]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:\\Users\\Francesco\\Desktop\\Capstone\\codes\\credentials.json"

In [4]:
start_date='2020-01-26'
end_date='2020-01-27'

In [5]:
cred_path='C:\\Users\\Francesco\\Desktop\\Capstone\\codes\\credentials.json'

In [22]:
import json
def read_config(config):
    # config is the name/path to your config file
    with open(config, 'r') as infile:
          config = json.load(infile)

    return(config)


config = read_config("C:\\Users\\Francesco\\Desktop\\Capstone\\codes\\cred.json")
print(config)
print(config['web']["client_id"])
#id, secret = config["client_id"], config['client_secret']

{'web': {'client_id': '35418705226-hvsvanmat91i1ifvjpd03egk4su9k7o2.apps.googleusercontent.com', 'project_id': 'nimble-valve-257105', 'auth_uri': 'https://accounts.google.com/o/oauth2/auth', 'token_uri': 'https://oauth2.googleapis.com/token', 'auth_provider_x509_cert_url': 'https://www.googleapis.com/oauth2/v1/certs', 'client_secret': 'ejXDs5ywmrxKUDeJb9DaO87g'}}
35418705226-hvsvanmat91i1ifvjpd03egk4su9k7o2.apps.googleusercontent.com


In [8]:
#credentials = service_account.Credentials.from_service_account_file(
#            'C:\\Users\\Francesco\\Desktop\\Capstone\\codes\\cred.json')

In [23]:
def get_gcp_creds(key_path):
    """
    Return GCP credentials from local .json
    
    Parameters
    ----------
    key_path: str
        Path to local credential .json
    """
    try:
        credentials = service_account.Credentials.from_service_account_file(
            key_path,
            scopes = ["https://www.googleapis.com/auth/cloud-platform"]
        )
    except:
        credentials = None
    return credentials;

In [24]:
def gbq_query(query,cred, query_params=None):
    """
    Run a query against Google Big Query, returning a pandas dataframe of the result.

    Parameters
    ----------
    query: str
        The query string
    cred: obj
        Credential class instance
    
    query_params: list, optional
        The query parameters to pass into the query string
    """
    client = bigquery.Client(credentials=cred)
    job_config = bigquery.QueryJobConfig()
    job_config.query_parameters = query_params
    return client.query(query, job_config=job_config).to_dataframe()

In [25]:
#Query and query_parameters
query1 = """
Select 
  TIMESTAMP(l.CloseTime) as `TimeStamp`,
  t.Account,t.Destination,t.Fee,t.TxnSignature,t.AmountXRP,t.LedgerIndex,t.TransactionType
FROM
  `xrpledgerdata.fullhistory.transactions`t
JOIN
  `xrpledgerdata.fullhistory.ledgers` l
  on t.LedgerIndex=l.LedgerIndex
where t.TransactionResult = "tesSUCCESS"

AND TIMESTAMP(l.CloseTime) >=TIMESTAMP(@start_date)
AND TIMESTAMP(l.CloseTime) <=TIMESTAMP(@end_date)

LIMIT 1000000

"""
query_params1 = [
    bigquery.ScalarQueryParameter("start_date", "STRING", start_date),
    bigquery.ScalarQueryParameter("end_date", "STRING", end_date)
]

In [27]:
a = gbq_query(query1,get_gcp_creds(cred_path),query_params1)


In [26]:
get_gcp_creds(cred_path)

In [30]:
a.describe()

,Fee,LedgerIndex
count,7.191830e+05,7.191830e+05
mean,1.200839e+03,5.299983e+07
std,2.027118e+04,6.509140e+03
min,1.000000e+01,5.298859e+07
25%,1.200000e+01,5.299416e+07
50%,1.200000e+01,5.299980e+07
75%,1.200000e+01,5.300551e+07
max,8.252806e+06,5.301101e+07


In [31]:
w2 = datetime.now()
w1 = timedelta(days=7)
start = w2-w1
end = w2
print(start)
print(end)

2020-01-27 21:47:41.572469
2020-02-03 21:47:41.572469


In [32]:
print(end.day)
print(start.year)

3
2020


In [32]:
tx_max = 5
ledger_max = 53127223
account_max = 'blablabla'

In [36]:
tx = a.groupby(['LedgerIndex']).count().Account

In [111]:
b = a.groupby(['Account']).count().sort_values(by=['Destination'], ascending=False)[0:1]
b

,TimeStamp,Destination,Fee,TxnSignature,AmountXRP,LedgerIndex,TransactionType
Account,,,,,,,
rAPERVgXZavGgiGv6xBgtiZurirW2yAmY,4996,4996,4996,4996,4996,4996,4996


In [122]:
tx_max = tx.max()
ledger_max = tx.idxmax()
account_max = b.index[0]
print(tx_max)
print(ledger_max)
print(account_max)

635
52996068
rAPERVgXZavGgiGv6xBgtiZurirW2yAmY


In [123]:
url = "https://livenet.xrpl.org/ledgers/" + str(ledger_max) 

In [124]:
document = Document()

document.add_heading('Weekly report', 0)

p = document.add_paragraph('Here\'s a few things that happened on the XRP ledger in the week from ' 
                           + str(start.year) + '-' + str(start.month) + '-'+ str(start.day) 
                           + ' to\n' + str(end.year) + '-'+ str(end.month) + '-'+ str(end.day))
document.add_paragraph(
    'Highest # of transactions per ledger was ' + str(tx_max) + ' for ledger ' + str(ledger_max) + "\n" + url+ "\n", style='List Number')

document.add_paragraph(
    'The account with the highest # of payments was ' + str(account_max), style='List Number')


#document.add_picture('write-word-using-python.jpg', width=Inches(1.25))

document.add_page_break()

document.save('C:\\Users\\Francesco\\Desktop\\Capstone\\codes\\weekly_report'+str(start.month)+str(start.day)+'_'
              +str(end.month)+str(end.day)+'.docx')

In [125]:
print('weekly_report'+str(start.month)+str(start.day)+'_'
              +str(end.month)+str(end.day)+'.docx')

weekly_report127_23.docx


In [7]:
##############################################
#useful documentation 


document = Document()

document.add_heading('Weekly report', 0)

p = document.add_paragraph('A plain paragraph having some ')
p.add_run('bold').bold = True
p.add_run(' and some ')
p.add_run('italic.').italic = True

document.add_heading('Heading, level 1', level=1)

document.add_paragraph('Intense quote', style='Intense Quote')

document.add_paragraph(
    'first item in unordered list', style='List Bullet'
)

document.add_paragraph(
    'first item in ordered list', style='List Number'
)

#document.add_picture('write-word-using-python.jpg', width=Inches(1.25))


recordset = [
    {
        "id" : 1,
        "qty": 2,
        "desc": "New item"
    },
    {
        "id" : 2,
        "qty": 2,
        "desc": "New item"
    },
    {
        "id" : 3,
        "qty": 2,
        "desc": "New item"
    }
]
#print(recordset[0]['id'])
table = document.add_table(rows=1, cols=3)
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Id'
hdr_cells[1].text = 'Quantity'
hdr_cells[2].text = 'Description'
for item in recordset:
	#print(item)
	row_cells = table.add_row().cells
	row_cells[0].text = str(item['id'])
	row_cells[1].text = str(item['qty'])
	row_cells[2].text = str(item['desc'])

document.add_page_break()

document.save('C:\\Users\\Francesco\\Desktop\\Capstone\\codes\\weekly_report.docx')

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Francesco\\Desktop\\Capstone\\codes\\weekly_report.docx'

In [6]:
import textract
text = textract.process("C:\\Users\\Francesco\\Desktop\\Capstone\\codes\\weekly_report.docx")
text

b'Document Title\n\nA plain paragraph having some bold and some italic.\n\nHeading, level 1\n\nIntense quote\n\nfirst item in unordered list\n\nfirst item in ordered list\n\nId\n\nQuantity\n\nDescription\n\n1\n\n2\n\nNew item\n\n2\n\n2\n\nNew item\n\n3\n\n2\n\nNew item'